In [1]:
#本案例中利用前三个word预测最后一个word
# b =4  m=max_len=n_step=3     v=n_class=13    h=n_hidden=5

# input = (b,m)  ========>( np.eye(v)[input]) 操作  ) =====>(b,m,v)
# X.transpose(0, 1)         X :(b,m,v)===>(m,b,v)



#      X :(m,b,v)
#     介绍num_layers:  https://discuss.pytorch.org/t/what-is-num-layers-in-rnn-module/9843/2   
# hidden :[num_layers(=1) * num_directions(=1) , b,  h]

# self.rnn = nn.RNN(input_size=v, hidden_size=h)           #(v,h)
#RNN 操作：     outputs, hidden = self.rnn(X, hidden)         #self.rnn(  (m,b,v),(1,b,h) )

 # 得到的outputs : [m, b, num_directions(=1) * h] 
# 得到的hidden : [num_layers(=1) * num_directions(=1), b, h]







# 然后用output中的最后一层：outputs = outputs[-1]              # [b, num_directions(=1) * h]=(b,h)
    
 # (b, num_directions(=1) * h )  *    (num_directions(=1) * h ,v)= (b,h)*(h,v) = (b,v)
#(b,v)与实际的y做softmax得到loss 

 word_list= ['coffee', 'memory', 'trully', 'cat', 'like', 'really', 'i', 'dog', 'hate', 'love', 'very', 'milk', 'do']
 word_dict= {'coffee': 0, 'memory': 1, 'trully': 2, 'cat': 3, 'like': 4, 'really': 5, 'i': 6, 'dog': 7, 'hate': 8, 'love': 9, 'very': 10, 'milk': 11, 'do': 12}
 number_dict= {0: 'coffee', 1: 'memory', 2: 'trully', 3: 'cat', 4: 'like', 5: 'really', 6: 'i', 7: 'dog', 8: 'hate', 9: 'love', 10: 'very', 11: 'milk', 12: 'do'}
n_class =  13


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

sentences = [ "i really like dog", "i  do love coffee", "i very hate milk","i trully memory cat"]

word_list = " ".join(sentences).split()

word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)      #v

print(' word_list=',word_list)
print(' word_dict=',word_dict)
print(' number_dict=',number_dict)
print("n_class = " ,len(word_list))


 word_list= ['coffee', 'memory', 'trully', 'cat', 'like', 'really', 'i', 'dog', 'hate', 'love', 'very', 'milk', 'do']
 word_dict= {'coffee': 0, 'memory': 1, 'trully': 2, 'cat': 3, 'like': 4, 'really': 5, 'i': 6, 'dog': 7, 'hate': 8, 'love': 9, 'very': 10, 'milk': 11, 'do': 12}
 number_dict= {0: 'coffee', 1: 'memory', 2: 'trully', 3: 'cat', 4: 'like', 5: 'really', 6: 'i', 7: 'dog', 8: 'hate', 9: 'love', 10: 'very', 11: 'milk', 12: 'do'}
n_class =  13


In [3]:
# TextRNN Parameter
batch_size = len(sentences)      #b=4
n_step = 3 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        # print(input)  
        #[[11, 6, 8]
        # print(np.eye(n_class)[input])        #数组中的数字  转化为onehot数组     
        '''
        [   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
            [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
            [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] ] '''
        input_batch.append(np.eye(n_class)[input])
        
        target = word_dict[word[-1]]
        target_batch.append(target)

    return input_batch, target_batch

# to Torch.Tensor
#input_batch  是每个sentence中[:-1]个word
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.Tensor(input_batch))
print(input_batch.size())    #(b,m,v)=([4, 3, 13])

target_batch = Variable(torch.LongTensor(target_batch))
print(target_batch.size())    #(b) =([4])



torch.Size([4, 3, 13])
torch.Size([4])


In [4]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()

        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)   #(v,h)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))  #(h,v)
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, hidden, X):
        X = X.transpose(0, 1)                       # X :(b,m,v)===>(m,b,v)=(3,4,13)   transpose(0,1)表示第一个和第二个维度互换
        outputs, hidden = self.rnn(X, hidden)       # hidden : [num_layers * num_directions, b, h]
        # outputs : [m, b, num_directions(=1) * h]
        # hidden : [num_layers(=1) * num_directions(=1), b, h]
        outputs = outputs[-1] # [b, num_directions(=1) * h]
        model = torch.mm(outputs, self.W) + self.b # model :(h,v)*(b,h) =(b,v)
        return model

model = TextRNN()
# print(model)
'''
TextRNN(
  (rnn): RNN(13, 5)
)
'''
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(1):
    optimizer.zero_grad()

    # hidden : [num_layers * num_directions, b, h]
    hidden = Variable(torch.zeros(1, batch_size, n_hidden))
    # input_batch : [b, n_step, n_class]
    output = model(hidden, input_batch)   #(b,v)

    # output : [b, v],          target_batch : [b] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.077891


Epoch: 2000 cost = 0.020939


Epoch: 3000 cost = 0.008143


Epoch: 4000 cost = 0.003789


Epoch: 5000 cost = 0.001957


In [5]:
input = [sen.split()[:-1] for sen in sentences]

# Predict
hidden = Variable(torch.zeros(1, batch_size, n_hidden))
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:-1] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'really', 'like'], ['i', 'do', 'love'], ['i', 'very', 'hate'], ['i', 'trully', 'memory']] -> ['dog', 'coffee', 'milk', 'cat']
